# Double Heston Model

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

## Vanilla Option Pricing

In [ ]:
# Double Heston characteristic function
def double_heston_charfunc(u, params):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]

    # Parameters of variance process 1
    v01    = params["v01"]
    kappa1 = params["kappa1"]
    theta1 = params["theta1"]
    sigma1 = params["sigma1"]
    rho1   = params["rho1"]

    # Parameters of variance process 2
    v02    = params["v02"]
    kappa2 = params["kappa2"]
    theta2 = params["theta2"]
    sigma2 = params["sigma2"]
    rho2   = params["rho2"]

    iu = 1j * u

    # Helper to compute C and D for each variance component
    def _terms(kappa, theta, sigma, rho):
        alpha = -0.5 * (u**2 + iu)
        beta  = kappa - rho * sigma * iu
        gamma = 0.5 * sigma**2
        d     = np.sqrt(beta**2 - 4.0 * alpha * gamma)
        g     = (beta - d) / (beta + d)
        exp_dT = np.exp(-d * T)

        C = (kappa * theta / gamma) * ((beta - d) * T - 2.0 * np.log((1.0 - g * exp_dT) / (1.0 - g)))
        D = (beta - d) / gamma * ((1.0 - exp_dT) / (1.0 - g * exp_dT))
        return C, D

    # Compute C and D for both factors
    C1, D1 = _terms(kappa1, theta1, sigma1, rho1)
    C2, D2 = _terms(kappa2, theta2, sigma2, rho2)

    # Combine
    C_total = (r - q) * iu * T + C1 + C2
    D_total = D1 * v01 + D2 * v02

    return np.exp(C_total + D_total + iu * np.log(S0 * np.exp(-q * T)))


# FFT-based pricing using any characteristic function
def fft_price_call_put(params, N=10000, U_max=1000):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    # Ensure even
    if N % 2 == 1:
        N += 1

    # FFT grid
    u  = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    # Characteristic functions
    phi_u     = double_heston_charfunc(u, params)
    phi_u_im1 = double_heston_charfunc(u - 1j, params)
    phi_im1   = double_heston_charfunc(-1j, params)

    # Integrands
    integrand_P1 = np.real(np.exp(-1j * u * lnK) * phi_u_im1 / (1j * u * phi_im1))
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    # Simpson's rule weights
    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    # Probabilities
    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1)
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    # Prices
    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put  = K * np.exp(-r * T) * (1 - P2) - S0 * np.exp(-q * T) * (1 - P1)
    return call, put


In [6]:
params = {
    "S0": 3795.74,
    "K" : 3795.74,
    "r":  0.014,
    "q": 0.015,
    "v01": 0.0112,
    "kappa1": 0.2183,
    "theta1": 0.0601,
    "sigma1": 0.3171,
    "rho1": -0.5250,
    "v02": 0.0112,
    "kappa2": 0.2183,
    "theta2": 0.0601,
    "sigma2": 0.3171,
    "rho2": -0.5250,
    "T": 1.0
}
call, put = fft_price_call_put(params)
print(f"Call Price: {call:.4f}")
print(f"Put Price: {put:.4f}")

Call Price: 352.3063
Put Price: 356.0474


## Data Extraction

In [10]:
# 1) On définit d’abord les maturités (index) et les niveaux de strikes (colonnes)
maturities = ['T', '1M', '2M', '3M', '6M', '9M', '1Y', '18M', '2Y', '3Y', '4Y', '5Y', '7Y', '10Y']
strikes_pct = ['80%', '90%', '95%', '97.5%', '100%', '102.5%', '105%', '110%', '120%']

# 2) On crée ensuite la liste des listes « raw » correspondant aux lignes du tableau
data = [
    # Strike prices (ligne 'T')
    [3036.59, 3416.17, 3605.95, 3700.85, 3795.74, 3890.63, 3985.53, 4175.31, 4554.89],
    # Valeurs pour 1M
    [25.29,   19.15,   15.35,   13.70,   12.25,   11.04,   10.71,   11.46,   13.56],
    # 2M
    [24.28,   18.46,   15.81,   14.58,   13.43,   12.39,   11.67,   11.52,   12.55],
    # 3M
    [23.03,   18.08,   15.96,   14.96,   14.00,   13.10,   12.38,   11.83,   12.06],
    # 6M
    [21.93,   18.24,   16.61,   15.85,   15.13,   14.47,   13.86,   12.87,   12.40],
    # 9M
    [21.22,   18.35,   17.05,   16.41,   15.79,   15.20,   14.65,   13.71,   12.72],
    # 1Y
    [21.14,   18.61,   17.48,   16.93,   16.39,   15.86,   15.35,   14.43,   13.28],
    # 18M
    [21.15,   19.15,   18.18,   17.71,   17.24,   16.79,   16.36,   15.58,   14.39],
    # 2Y
    [21.03,   19.33,   18.51,   18.10,   17.71,   17.33,   16.97,   16.30,   15.22],
    # 3Y
    [21.19,   19.68,   18.95,   18.59,   18.25,   17.91,   17.59,   16.99,   15.98],
    # 4Y
    [21.29,   19.99,   19.36,   19.06,   18.77,   18.49,   18.22,   17.72,   16.84],
    # 5Y
    [21.52,   20.36,   19.82,   19.56,   19.31,   19.06,   18.82,   18.38,   17.60],
    # 7Y
    [22.14,   21.18,   20.74,   20.53,   20.33,   20.13,   19.94,   19.58,   18.93],
    # 10Y
    [23.17,   22.41,   22.06,   21.89,   21.73,   21.57,   21.42,   21.14,   20.62],
]

# 3) On construit le DataFrame
df = pd.DataFrame(data, index=maturities, columns=strikes_pct)

# 4) Optionnel : convertir en float (ici déjà float)
df = df.astype(float)

print(df)


         80%      90%      95%    97.5%     100%   102.5%     105%     110%  \
T    3036.59  3416.17  3605.95  3700.85  3795.74  3890.63  3985.53  4175.31   
1M     25.29    19.15    15.35    13.70    12.25    11.04    10.71    11.46   
2M     24.28    18.46    15.81    14.58    13.43    12.39    11.67    11.52   
3M     23.03    18.08    15.96    14.96    14.00    13.10    12.38    11.83   
6M     21.93    18.24    16.61    15.85    15.13    14.47    13.86    12.87   
9M     21.22    18.35    17.05    16.41    15.79    15.20    14.65    13.71   
1Y     21.14    18.61    17.48    16.93    16.39    15.86    15.35    14.43   
18M    21.15    19.15    18.18    17.71    17.24    16.79    16.36    15.58   
2Y     21.03    19.33    18.51    18.10    17.71    17.33    16.97    16.30   
3Y     21.19    19.68    18.95    18.59    18.25    17.91    17.59    16.99   
4Y     21.29    19.99    19.36    19.06    18.77    18.49    18.22    17.72   
5Y     21.52    20.36    19.82    19.56    19.31    

## Model Calibration

In [11]:
# --- 1) Black–Scholes call price for reconstructing market prices from vols ---

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

# --- 2) Double Heston characteristic & FFT pricing functions must be defined or imported here ---
# from double_heston import double_heston_charfunc, fft_price_call_put

# --- 3) Extraction of market data ---------------------------------------------------
# df: DataFrame with index of maturities ('1M', '3M', ..., 'T') and columns of strikes ('80%',..., '120%')

# Known constants
S0 = 3795.74
r  = 0.014    # 1.40% contin
q  = 0.015    # 1.50% contin

# Convert maturity labels to years
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(f"Unknown maturity: {label}")

# Build market data list: (T, K, call_price)
market_data = []
strikes_abs = df.loc['T']
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K       = strikes_abs[pct]
        vol_imp = df.loc[mat, pct] / 100.0
        price   = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data,
                       dtype=[('T', float), ('K', float), ('price', float)])

# --- 4) Parameter packing for double Heston -----------------------------------------

def pack_params(x):
    return {
        "S0":    S0,
        "r":     r,
        "q":     q,
        # Heston factor 1
        "v01":   x[0],
        "kappa1":x[1],
        "theta1":x[2],
        "sigma1":x[3],
        "rho1":  x[4],
        # Heston factor 2
        "v02":   x[5],
        "kappa2":x[6],
        "theta2":x[7],
        "sigma2":x[8],
        "rho2":  x[9],
    }

# Residuals between model and market

def residuals(x):
    params_base = pack_params(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = fft_price_call_put(params, N=2000, U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)

# --- 5) Calibration via non-linear least squares ------------------------------------
# Initial guess for [v01, kappa1, theta1, sigma1, rho1, v02, kappa2, theta2, sigma2, rho2]
x0 = np.array([
    0.04,    # v01
    1.0,     # kappa1
    0.04,    # theta1
    0.5,     # sigma1
    -0.5,    # rho1
    0.04,    # v02
    1.0,     # kappa2
    0.04,    # theta2
    0.5,     # sigma2
    -0.5,    # rho2
])
# Bounds: v's, kappas, thetas, sigmas >= 0; rhos in [-0.999, 0.999]
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999,
      1e-4, 1e-4, 1e-4, 1e-4, -0.999]
ub = [2.0, 10.0, 2.0, 5.0, 0.999,
      2.0, 10.0, 2.0, 5.0, 0.999]

opt = least_squares(
    residuals,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=300
)

# --- 6) Display results -------------------------------------------------------------
opt_x = opt.x
print("Calibrated double Heston parameters:")
print(f" Factor 1: v01={opt_x[0]:.4f}, kappa1={opt_x[1]:.4f}, theta1={opt_x[2]:.4f}, sigma1={opt_x[3]:.4f}, rho1={opt_x[4]:.4f}")
print(f" Factor 2: v02={opt_x[5]:.4f}, kappa2={opt_x[6]:.4f}, theta2={opt_x[7]:.4f}, sigma2={opt_x[8]:.4f}, rho2={opt_x[9]:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.4656e+06                                    1.50e+07    
       1              2         4.1351e+05      7.05e+06       1.43e+00       1.73e+06    
       2              3         7.2778e+04      3.41e+05       4.64e-01       1.77e+05    
       3              4         2.3338e+04      4.94e+04       4.77e-01       4.65e+04    
       4              5         1.3573e+04      9.77e+03       8.93e-02       3.60e+05    
       5              6         1.2405e+04      1.17e+03       1.09e+00       1.64e+05    
       6              7         9.1102e+03      3.29e+03       7.69e-02       2.26e+04    
       7              9         8.1537e+03      9.56e+02       3.50e-01       1.83e+04    
       8             10         7.0125e+03      1.14e+03       4.76e-01       9.40e+04    
       9             11         6.3633e+03      6.49e+02       1.10e-01       2.16e+03    

In [12]:
print("Prix Black-Sholes, M=1Y, S=100%:", bs_call_price(3795.74,3795.74,0.014,0.015,0.1639,1.0))

params_calibrated = {
    "S0": 3795.74,
    "K" : 3795.74,
    "r":  0.014,
    "q": 0.015,
    "v01": 0.0074,
    "kappa1": 4.6367,
    "theta1": 0.0117,
    "sigma1": 0.3948,
    "rho1": -0.6750,
    "v02": 0.0001,
    "kappa2": 0.1555,
    "theta2": 0.1052,
    "sigma2": 1.0370,
    "rho2": -0.9758,
    "T": 1.0
}
call, put = fft_price_call_put(params_calibrated)
print(f"Call Price (calibrated): {call:.4f}")

Prix Black-Sholes, M=1Y, S=100%: 242.4783779478173
Call Price (calibrated): 242.6778
